In [9]:
from __future__ import annotations

from PIL import Image
from pyrr import Matrix44

import moderngl as mgl
import numpy as np

import pygame
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import serial
import serial.tools.list_ports
import time
import math 

start = time.time()

pygame.init()
screen = pygame.display.set_mode([1920, 1080], pygame.RESIZABLE)
clock = pygame.time.Clock()
font = pygame.font.SysFont("Arial", 32)

fpslist = []
def pilImageToSurface(pilImage):
    return pygame.image.fromstring(
        pilImage.tobytes(), pilImage.size, pilImage.mode).convert()

        
def show_fbo(fbo: mgl.Framebuffer, size: tuple[int], color_mode: str) -> None:
    img = Image.frombytes(color_mode, size, fbo.read(components=len(color_mode)))
    img = img.transpose(Image.FLIP_TOP_BOTTOM)
    pygameSurface = pilImageToSurface(img)
    screen.fill(0)
    screen.blit(pygameSurface, pygameSurface.get_rect(center = (1400, 490)))

def update_fps():
	fps = int(clock.get_fps())
	fpslist.append(fps)
	fpstring = str(fps)
	fps_text = font.render(fpstring, 1, pygame.Color("coral"))
	return fps_text

def portsDetection():
     print("waiting for connection...")
     ports = list(serial.tools.list_ports.comports())
     for p in ports:
          print(p.manufacturer)
          if "Arduino" in p.manufacturer:
               print("This is an Arduino!")
               return(serial.Serial(str(p.device), 115200, timeout=10))
     raise Exception("No device Found")

ser = "none"    
SIZE = WIDTH, HEIGHT = int(1080), int(1080)
COLOR_MODE = "RGBA"
ctx = mgl.create_context(standalone=True)
fbo = ctx.simple_framebuffer(SIZE, components=len(COLOR_MODE))
fbo.use()

vbo = ctx.buffer(np.array([
    # x     y    z     u    v  
     1,  1, 0.0,  1., 1.,
     1, -1, 0.0,  1., -1.,
    -1, -1, 0.0,  -1., -1.,
    -1,  1, 0.0,  -1., 1.,
], dtype=np.float32))

ibo = ctx.buffer(np.array([
    0, 1, 3,
    1, 2, 3,
], dtype=np.int32))

In [10]:
VS = """
#version 330

in vec3 in_vert;
in vec2 in_uv;

out vec2 fragCoord;

void main() {
    gl_Position = vec4(in_vert, 1.0f);
    fragCoord = in_uv;
}
"""
run = True
up = True
frame = 0

alpha = [0,0,0,0]
errcpt = 0
while run:
  frame += 1
  clock.tick(0)
  screen.blit(update_fps(), (10,0))
  try:
    if (ser.inWaiting() > 0):
      strval = ser.readline().decode().strip()
      if "A" in strval:
        alpha[0] = ((int(strval.split("A")[1].split("B")[0])*2.2)/1024)-0.9
        alpha[1] = ((int(strval.split("B")[1].split("C")[0])*2.2)/1024)-0.9
        alpha[2] = ((int(strval.split("C")[1].split("D")[0])*2.2)/1024)-0.9
        alpha[3] = ((int(strval.split("D")[1])*2.2)/1024)-0.9
    errcpt = 0
  except Exception:
          errcpt += 1
          if (errcpt >= 5):
               print("Connection error, please check cables")
               text = font.render("waiting for connection...", 1, pygame.Color("red"))
               screen.blit(update_fps(), (10,20))
               found = False
               while found == False:
                    time.sleep(1)
                    try:
                         ser = portsDetection()
                         found = True
                    except Exception:
                         found = False
          pass
  
  pygame.display.update()
  for event in pygame.event.get():
      if event.type == pygame.QUIT:
        run = False
  FS = """
  #version 330
  out vec4 fragColor; 
  in vec2 fragCoord;

  // Constants
  const int MAX_MARCHING_STEPS = 255;
  const float MIN_DIST = 0.5;
  const float MAX_DIST = 200.0;
  const float PRECISION = 0.01;
  const float EPSILON = 0.05;
  const float PI = 3.14159265359;


  float satFactor=0.5;
 float movingSpeed=1.0;
      
  float alpha11 = 0.;
  float alpha12 = 0.;
  float alpha13 = 0.;

  float alpha21 = 0.;
  float alpha22 = 0.;
  float alpha23 = 0.;

  float alpha31 = 0.;
  float alpha32 = 0.;
  float alpha33 = 0.;

  float alpha41 = 0.;
  float alpha42 = 0.;
  float alpha43 = 0.;

  float L11 = 1.;
  float L12 = 0.84;
  float L13 = 0.92;

  float L21 = 1.08;
  float L22 = 1.04;
  float L23 = 1.;

  float L31 = 0.9;
  float L32 = 0.9;
  float L33 = 1.;

  float L41 = 0.64;
  float L42 = 0.6;
  float L43 = 0.84;


  vec3 center1 = vec3(0.,-1.,1.);
  vec3 P11 = vec3(0.,0.,0.);
  vec3 P12 = vec3(0.,0.,0.);
  vec3 P13 = vec3(0.,0.,0.);

  vec3 center2 = vec3(0.,-1.,0.5);
  vec3 P21 = vec3(0.,0.,0.);
  vec3 P22 = vec3(0.,0.,0.);
  vec3 P23 = vec3(0.,0.,0.);

  vec3 center3 = vec3(0.,-1.,0.);
  vec3 P31 = vec3(0.,0.,0.);
  vec3 P32 = vec3(0.,0.,0.);
  vec3 P33 = vec3(0.,0.,0.);

  vec3 center4 = vec3(0.,-1.,-0.5);
  vec3 P41 = vec3(0.,0.,0.);
  vec3 P42 = vec3(0.,0.,0.);
  vec3 P43 = vec3(0.,0.,0.);

  mat3 rotateX(float theta) {
      float c = cos(theta);
      float s = sin(theta);
      return mat3(
          vec3(1, 0, 0),
          vec3(0, c, -s),
          vec3(0, s, c)
      );
  }

  mat3 rotateY(float theta) {
      float c = cos(theta);
      float s = sin(theta);
      return mat3(
          vec3(c, 0, s),
          vec3(0, 1, 0),
          vec3(-s, 0, c)
      );
  }

  mat3 rotateZ(float theta) {
      float c = cos(theta);
      float s = sin(theta);
      return mat3(
          vec3(c, -s, 0),
          vec3(s, c, 0),
          vec3(0, 0, 1)
      );
  }

  // Identity matrix.
  mat3 identity() {
      return mat3(
          vec3(1, 0, 0),
          vec3(0, 1, 0),
          vec3(0, 0, 1)
      );
  }

  struct Surface {
      float sd; // signed distance value
      vec3 col; // color
  };
  float sdRoundBox( vec3 p, vec3 b, float r )
  {
    vec3 q = abs(p) - b;
    return length(max(q,0.0)) + min(max(q.x,max(q.y,q.z)),0.0) - r;
  }
  Surface sdBox( vec3 p, vec3 b, vec3 offset, vec3 col, mat3 transform)
  {

    p = (p - offset) * transform; // apply transformation matrix
    vec3 q = abs(p) - b;
    float d = length(max(q,0.0)) + min(max(q.x,max(q.y,q.z)),0.0);
    return Surface(d, col);
  }

  Surface sdFloor(vec3 p, vec3 col) {
    float d = p.y + 1.;
    return Surface(d, col);
  }

  float smin(float a, float b, float k) {
      float h = clamp(0.5 + 0.5*(a-b)/k, 0.0, 1.0);
      return mix(a, b, h) - k*h*(1.0-h);
  }

  Surface minWithColor(Surface obj1, Surface obj2) {

    return Surface(smin(obj1.sd,obj2.sd, 0.1), obj2.col);
  }

  Surface sdCapsule( vec3 p, vec3 a, vec3 b, float r , vec3 col)
  {
    vec3 pa = p - a, ba = b - a;
    float h = clamp( dot(pa,ba)/dot(ba,ba), 0.0, 1.0 );
    return Surface(length( pa - ba*h ) - r, col);
  }

  Surface sdRoundBox( vec3 p, vec3 b, float r , vec3 col)
{
  vec3 q = abs(p) - b;
  return Surface(length(max(q,0.0)) + min(max(q.x,max(q.y,q.z)),0.0) - r,col);
}

  Surface sdScene2(vec3 p) {
    Surface co = sdCapsule(p, vec3(0.), vec3(1.), 1., vec3(1.,1.,1.));
    return co;
  }
  vec3 color = vec3(0.5,0.5,0.6);

  Surface sdScene(vec3 p) {
    Surface co = sdCapsule(p, center1, P11, 0.2*1.35, color);
    co = minWithColor(co, sdCapsule(p, P11, P12, 0.18*1.35, color));
    co = minWithColor(co, sdCapsule(p, P12, P13, 0.16*1.35, color));
    
    co = minWithColor(co, sdCapsule(p, center2, P21, 0.19*1.35, color));
    co = minWithColor(co, sdCapsule(p, P21, P22, 0.18*1.35, color));
    co = minWithColor(co, sdCapsule(p, P22, P23, 0.175*1.35, color));
    
    co = minWithColor(co, sdCapsule(p, center3, P31, 0.18*1.35, color));
    co = minWithColor(co, sdCapsule(p, P31, P32, 0.17*1.35, color));
    co = minWithColor(co, sdCapsule(p, P32, P33, 0.16*1.35, color));
    
    co = minWithColor(co, sdCapsule(p, center4, P41, 0.16*1.35, color));
    co = minWithColor(co, sdCapsule(p, P41, P42, 0.15*1.35, color));
    co = minWithColor(co, sdCapsule(p, P42, P43, 0.15*1.35, color));
    co = minWithColor(co, sdRoundBox(p + vec3(-0.1,2.1,-0.2), vec3(0.000000001,1.,0.75), 0.3, color));
    return co;
  }

  Surface rayMarch(vec3 ro, vec3 rd, float start, float end) {
    float depth = start;
    Surface co; // closest object

    for (int i = 0; i < MAX_MARCHING_STEPS; i++) {
      vec3 p = ro + depth * rd;
      co = sdScene(p);
      depth += co.sd;
      if (co.sd < PRECISION || depth > end) break;
    }
    
    co.sd = depth;
    
    return co;
  }

  vec3 calcNormal(in vec3 p) {
      vec2 e = vec2(1, -1) * EPSILON;
      return normalize(
        e.xyy * sdScene(p + e.xyy).sd +
        e.yyx * sdScene(p + e.yyx).sd +
        e.yxy * sdScene(p + e.yxy).sd +
        e.xxx * sdScene(p + e.xxx).sd);
  }


  mat3 camera(vec3 cameraPos, vec3 lookAtPoint) {
    vec3 cd = normalize(lookAtPoint - cameraPos); // camera direction
    vec3 cr = normalize(cross(vec3(0, 1, 0), cd)); // camera right
    vec3 cu = normalize(cross(cd, cr)); // camera up
    
    return mat3(-cr, cu, -cd);
  }

  void main()
  {

  alpha11 = """ + str(alpha[0]) + """-4;
  alpha12 = """ + str(alpha[0]) + """ +1. + alpha11;
  alpha13 = """ + str(alpha[0]) + """ +1. + alpha12;
  P11 = vec3(L11*cos(alpha11),L11*sin(alpha11),0.) + center1;
  P12 = vec3(L12*cos(alpha12),L12*sin(alpha12),0.) + P11;
  P13 = vec3(L13*cos(alpha13),L13*sin(alpha13),0.) + P12;


  alpha21 = """ + str(alpha[1]) + """-4.;
  alpha22 = """ + str(alpha[1]) + """ +1. + alpha21;
  alpha23 = """ + str(alpha[1]) + """ +1. + alpha22;
  P21 = vec3(L21*cos(alpha21),L21*sin(alpha21),0.) + center2;
  P22 = vec3(L22*cos(alpha22),L22*sin(alpha22),0.) + P21;
  P23 = vec3(L23*cos(alpha23),L23*sin(alpha23),0.) + P22;

  alpha31 = """ + str(alpha[2]) + """-4.;
  alpha32 = """ + str(alpha[2]) + """+1. + alpha31;
  alpha33 = """ + str(alpha[2]) + """+1. + alpha32;
  P31 = vec3(L31*cos(alpha31),L31*sin(alpha31),0.) + center3;
  P32 = vec3(L32*cos(alpha32),L32*sin(alpha32),0.) + P31;
  P33 = vec3(L33*cos(alpha33),L33*sin(alpha33),0.) + P32;

  alpha41 = """ + str(alpha[3]) + """-4.;
  alpha42 = """ + str(alpha[3]) + """ +1. + alpha41;
  alpha43 = """ + str(alpha[3]) + """ +1. + alpha42;
  P41 = vec3(L41*cos(alpha41),L41*sin(alpha41),0.) + center4;
  P42 = vec3(L42*cos(alpha42),L42*sin(alpha42),0.) + P41;
  P43 = vec3(L43*cos(alpha43),L43*sin(alpha43),0.) + P42;
    vec2 uv = fragCoord;

    vec3 backgroundColor1 = vec3(0.1 ,0.1,0.12);

    vec3 col = vec3(0.);
    vec3 lp = vec3(""" + str(math.sin(time.time()/2)) + """, 0., """+ str(math.cos(time.time()/2))+"""); // lookat point (aka camera target)
    vec3 ro = vec3(0., 0., 0.); // ray origin that represents camera position
    
    float cameraRadius = 3.25;
    ro.x = cameraRadius * (lp.x + ro.x); // convert to polar 
    ro.z = cameraRadius * (lp.z + ro.z);
    ro.y = cameraRadius * (lp.y + ro.y);
    
    vec3 rd = camera(ro, lp) * normalize(vec3(uv, -1)); // ray direction

    Surface co = rayMarch(ro, rd, MIN_DIST, MAX_DIST); // closest object

    if (co.sd > MAX_DIST) {
      col =  backgroundColor1;
    } else {
      vec3 p = ro + rd * co.sd;
      vec3 normal = calcNormal(p);
      vec3 lightPosition = vec3(-1., 3., 0.);
      vec3 lightDirection = normalize(lightPosition - p);
      vec3 lightPosition3 = vec3(2, 1., 0.);
      vec3 lightDirection3 = normalize(lightPosition - p);
      vec3 lightPosition2 = vec3(-0, -4, 1);
      vec3 lightDirection2 = normalize(lightPosition2 - p);

      float dif = clamp(dot(normal, lightDirection), 0.7, 100.);
      float dif2 = clamp(dot(normal, lightDirection2), 0.7, 100.);
      float dif3 = clamp(dot(normal, lightDirection3), 0.7, 200.);

      col = (dif3 *dif * dif2 * (2*co.col))/1.2;
      //col = dif2 * co.col;
      //col = co.col;
    }

    fragColor = vec4(col, 1.0);
  }

  """
  prog = ctx.program(vertex_shader=VS, fragment_shader=FS)
  vao = ctx.vertex_array(prog, [(vbo, "3f 2f", "in_vert", "in_uv")], ibo)
  #ctx.clear(0.0, 0.0, 0.0, 1.0)
  vao.render(mode=mgl.TRIANGLES)
  show_fbo(fbo, SIZE, COLOR_MODE)
  #pygame.display.flip()
  if frame>30 and frame%10 == 0:
    fig = plt.figure(figsize=(8.4, 2.0), dpi=100)
    ax = fig.add_subplot(1, 1, 1)
    end = time.time()
    ax.plot(range(len(fpslist))[-600:-1],fpslist[-600:-1])
    #print("range " + str(len(range(frame)[-300:-1])))
    #print("list " + str(len(fpslist[-300:-1])))
    #print("--------")
    fig.set_facecolor('black')
    fig.canvas.draw()
    pil = Image.frombytes('RGB',fig.canvas.get_width_height(),fig.canvas.tostring_rgb())
    surf = pilImageToSurface(pil)
    matplotlib.pyplot.close()
  try:
    screen.blit(surf, surf.get_rect(center = (410, 800)))
  except Exception:
    pass
pygame.quit()

Connection error, please check cables
waiting for connection...
Arduino (www.arduino.cc)
This is an Arduino!
